<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/test_app_web3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -qqq web3

In [6]:
import numpy as np
import time
import requests, json

In [7]:
from web3 import Web3

# --- 1. تنظیمات اولیه با Alchemy ---

# آدرس نود اتریوم از طریق Alchemy
# توجه: کلید API خود را جایگزین 'YOUR_ALCHEMY_API_KEY' کنید
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
web3 = Web3(Web3.HTTPProvider(alchemy_url))

# بررسی اتصال به شبکه
if not web3.is_connected():
    print("dont connect through alchemy")
    exit()
else:
    print("connected")


# --- 2. اطلاعات استخر ---

# آدرس قرارداد استخر مورد نظر در یونی‌سواپ V3
# برای مثال، از استخر USDC/WETH با کارمزد 0.05% استفاده شده است
pool_address = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"


# --- 3. ABI قرارداد استخر ---

# ABI (Application Binary Interface) برای تعامل با توابع قرارداد هوشمند ضروری است.
# برای تابع tickSpacing()، تنها به بخش کوچکی از ABI کامل نیاز داریم.
pool_abi = [
    {
        "inputs": [],
        "name": "tickSpacing",
        "outputs": [
            {
                "internalType": "int24",
                "name": "",
                "type": "int24"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]


# --- 4. ایجاد نمونه قرارداد و فراخوانی تابع ---

# ایجاد یک نمونه از قرارداد استخر
pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)

try:
    # فراخوانی تابع 'tickSpacing' از قرارداد
    tick_spacing = pool_contract.functions.tickSpacing().call()

    # --- 5. نمایش نتیجه ---
    print("-" * 30)
    print(f"pool address: {pool_address}")
    print(f"pool Tick Spacing : {tick_spacing}")
    print("-" * 30)

except Exception as e:
    print(f"cause error : {e}")

connected
------------------------------
pool address: 0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082
pool Tick Spacing : 10
------------------------------


In [8]:
GRAPH_API_KEY='0f011cf8766c88189291aa0b95792828'
SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

In [9]:
def find_key(data, target_key):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            else:
                results.extend(find_key(value, target_key))
    elif isinstance(data, list):
        for item in data:
            results.extend(find_key(item, target_key))
    return results


In [10]:
def find_key_path(data, target_key):
    """
    جستجو در دیکشنری تو در تو برای یک کلید مشخص و برگرداندن
    مسیر root-to-leaf به صورت دیکشنری نگهدارنده ساختار.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                return {key: value}
            elif isinstance(value, (dict, list)):
                found = find_key_path(value, target_key)
                if found:
                    return {key: found}  # مسیر تا کلید هدف
    elif isinstance(data, list):
        for item in data:
            found = find_key_path(item, target_key)
            if found:
                return found
    return None


In [54]:
import requests, json

url = f"https://gateway.thegraph.com/api/{GRAPH_API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

pool_id = "0x5ae13baaef0620fdae1d355495dc51a17adb4082" # آدرس استخر (باید lowercase باشد)

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    createdAtTimestamp        # 1. زمان یونیکس ایجاد استخر (برای محاسبه عمر)
    tick
    liquidity                      # 2. تیک فعال فعلی (برای محاسبه ضریب تمرکز)
    feeTier                   # 3. نرخ کارمزد استخر (مثلاً 3000)
    feesUSD                   # 4. کارمزد کل انباشته تولید شده از ابتدا (صورت کسر APR)
    totalValueLockedUSD       # 5. کل نقدینگی قفل شده (TVL) به دلار (مخرج کسر APR)
    token0Price
    token1Price
    sqrtPrice
    token0 {{
      symbol                  # 6. نماد توکن 0 (PAXG)
      decimals                # 7. اعشار توکن 0 (برای تبدیل L به دلار)
    }}
    token1 {{
      symbol                  # 8. نماد توکن 1 (USDC)
      decimals                # 9. اعشار توکن 1 (برای تبدیل L به دلار)
    }}
  }}
}}
"""

r = requests.post(url, json={"query": query})
print(json.dumps(r.json(), indent=2))

{
  "data": {
    "pool": {
      "createdAtTimestamp": "1655264999",
      "feeTier": "500",
      "feesUSD": "507498.9568321475",
      "id": "0x5ae13baaef0620fdae1d355495dc51a17adb4082",
      "liquidity": "2641180442045884069",
      "sqrtPrice": "5107017504477634475339051",
      "tick": "-193000",
      "token0": {
        "decimals": "18",
        "symbol": "PAXG"
      },
      "token0Price": "0.0002406713946566963948268299621866352",
      "token1": {
        "decimals": "6",
        "symbol": "USDC"
      },
      "token1Price": "4155.043026307473102821356533130703",
      "totalValueLockedUSD": "11763038.71826353379283802673664965"
    }
  }
}


In [55]:
fee_tier=float(find_key(r.json(), "feeTier")[0])
tvl=float(find_key(r.json(), "totalValueLockedUSD")[0])
build_date=float(find_key(r.json(), "createdAtTimestamp")[0])
decimal0=float(find_key(r.json(), "decimals")[0])
decimal1=float(find_key(r.json(), "decimals")[1])
current_tick=float(find_key(r.json(), "tick")[0])
current_price=float(find_key(r.json(), "token1Price")[0])
sqrtprice=float(find_key(r.json(), "sqrtPrice")[0])

In [13]:
def calculate_price_by_tick(tick:int,decimal_0:int,decimal_1:int):
  return (10**(decimal_0-decimal_1))*(1.0001**tick)
  '''if decimal_0>decimal_1:
    return (10**(decimal_0-decimal_1))*(1.0001**tick)
  else:
    return (10**(decimal_1-decimal_0))*(1.0001**tick)'''

In [14]:
def calculate_tick_by_price(price:float,decimal_0:int,decimal_1:int):
  t=(np.log(price)-np.log((10**(decimal_0-decimal_1))))/(np.log(1.0001))
  if t>0:
    return np.ceil(t)
  else:
    return np.floor(t)
  """if decimal_0>decimal_1:
    t=(np.log(price)-np.log((10**(decimal_0-decimal_1))))/(np.log(1.0001))
    if t>0:
      return np.ceil(t)
    else:
      return np.floor(t)
  else:
    t=(np.log(price)-np.log((10**(decimal_1-decimal_0))))/(np.log(1.0001))
    if t>0:
      return np.floor(t)
    else:
      return np.ceil(t)"""


In [15]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:

    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)

    raw_price_math = normalized_sqrt_price ** 2

    decimal_adjustment = decimal_0 - decimal_1

    final_price = raw_price_math * (10 ** decimal_adjustment)

    return final_price

In [16]:
calculate_price_by_tick(current_tick,decimal0,decimal1)

4155.491708164091

In [17]:
calculate_tick_by_price(current_price,decimal0,decimal1)

np.float64(-192998.0)

In [18]:
calculate_price_from_sqrt_price(sqrtprice,decimal0,decimal1)

4155.764791580134

In [19]:
def pool_base_apr(inp_pool_id:str):


  urlv = f"https://gateway.thegraph.com/api/{GRAPH_API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

  pool_idv = inp_pool_id.lower()

  queryv = f"""
  {{
    pool(id: "{pool_idv}") {{
      id
      createdAtTimestamp        # 1. زمان یونیکس ایجاد استخر (برای محاسبه عمر)
      tick                      # 2. تیک فعال فعلی (برای محاسبه ضریب تمرکز)
      feeTier                   # 3. نرخ کارمزد استخر (مثلاً 3000)
      feesUSD                   # 4. کارمزد کل انباشته تولید شده از ابتدا (صورت کسر APR)
      totalValueLockedUSD       # 5. کل نقدینگی قفل شده (TVL) به دلار (مخرج کسر APR)
      token0Price
      token1Price
      sqrtPrice
      token0 {{
        symbol                  # 6. نماد توکن 0 (PAXG)
        decimals                # 7. اعشار توکن 0 (برای تبدیل L به دلار)
      }}
      token1 {{
        symbol                  # 8. نماد توکن 1 (USDC)
        decimals                # 9. اعشار توکن 1 (برای تبدیل L به دلار)
      }}
    }}
  }}
  """

  rv = requests.post(url, json={"query": query})

  #fee_tierv=float(find_key(rv.json(), "feeTier")[0])
  tvlv=float(find_key(rv.json(), "totalValueLockedUSD")[0])
  build_datev=float(find_key(rv.json(), "createdAtTimestamp")[0])
  #decimal0v=float(find_key(rv.json(), "decimals")[0])
  #decimal1v=float(find_key(rv.json(), "decimals")[1])
  #current_tickv=float(find_key(rv.json(), "tick")[0])
  #current_pricev=float(find_key(rv.json(), "token1Price")[0])
  #sqrtpricev=float(find_key(rv.json(), "sqrtPrice")[0])
  all_fee_usdv=float(find_key(rv.json(), "feesUSD")[0])
  #cpf=calculate_price_from_sqrt_price(sqrtpricev,decimal0v,decimal1v)
  now_time=time.time()
  days=(int(now_time)-build_datev)/(24*60*60)
  return (((all_fee_usdv/days)/tvl)+1)**365,((((all_fee_usdv/days)/tvl))*365)+1

In [20]:
base_apy,base_apr=pool_base_apr(pool_address)

In [21]:
import math

# نرخ پایه Uniswap (1.0001) به توان 0.5
Q_HALF = 1.0001 ** 0.5

def calculate_concentrated_apr_final(
    base_apr: float,
    current_tick: int,
    target_spacing: int,
    wide_ref_ticks: int = 500
) -> float:
    """
    محاسبه APR متمرکز نهایی با اعمال قدر مطلق بر ضریب تمرکز (Multiplier)
    برای جلوگیری از خطای ممیز شناور (Catastrophic Cancellation).
    """

    # --- 1. تعریف بازه تیک‌ها ---

    # بازه مرجع (500 تیک)
    ref_low_tick = current_tick - (wide_ref_ticks // 2)
    ref_high_tick = current_tick + (wide_ref_ticks // 2)

    # بازه هدف (Target Spacing)
    target_low_tick = current_tick
    target_high_tick = current_tick + target_spacing


    # --- 2. محاسبه Delta Sqrt Price برای بازه مرجع (Wide) ---

    # SqrtPrice = 1.0001 ** (tick / 2.0)
    sqrt_p_ref_high = 1.0001 ** (ref_high_tick / 2.0)
    sqrt_p_ref_low = 1.0001 ** (ref_low_tick / 2.0)

    # تفاضل (باید مثبت باشد، اما ممکن است به دلیل خطا منفی شود)
    delta_sqrt_price_wide = sqrt_p_ref_high - sqrt_p_ref_low


    # --- 3. محاسبه Delta Sqrt Price برای بازه هدف (Narrow) ---

    sqrt_p_target_high = 1.0001 ** (target_high_tick / 2.0)
    sqrt_p_target_low = 1.0001 ** (target_low_tick / 2.0)

    delta_sqrt_price_narrow = sqrt_p_target_high - sqrt_p_target_low


    # --- 4. محاسبه ضریب تمرکز ---
    if delta_sqrt_price_narrow == 0:
        return 0.0

    # اعمال قدر مطلق برای رفع خطای علامت منفی
    concentration_multiplier = abs(delta_sqrt_price_wide / delta_sqrt_price_narrow)

    # 5. محاسبه APR متمرکز نهایی
    apr_concentrated = base_apr * concentration_multiplier

    return apr_concentrated

In [22]:
calculate_concentrated_apr_final((base_apr-1),current_tick,10)

0.6238575940954716

In [23]:
calculate_concentrated_apr_final(0.18,current_tick,10)

8.997984594788194

In [24]:
calculate_concentrated_apr_final(0.12,current_tick,10)

5.998656396525464

In [25]:
def get_etherscan_block_by_timestamp(api_key: str,target_timestamp:int) -> int:
    """
    دریافت شماره بلاک اتریوم متناظر با 24 ساعت قبل از طریق API جدید Etherscan V2.
    """

    #print(f"زمان یونیکس 24 ساعت قبل: {timestamp_24h_ago}")

    api_url = "https://api.etherscan.io/v2/api"
    target_timestamp=int(target_timestamp)
    params = {
        "chain": "eth",
        "chainid": 1,
        "module": "block",
        "action": "getblocknobytime",
        "timestamp": target_timestamp,
        "closest": "before",
        "apikey": api_key
    }

    # نمایش URL کامل
    url_prepared = requests.Request('GET', api_url, params=params).prepare().url
    #print(f"URL ارسالی: {url_prepared}")

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        data = response.json()

        if response.status_code==200:
          if len(find_key(data,'result'))!=0:
            try:
              return int(find_key(data,'result')[0])
            except:
              return find_key(data,'result')[0]
          else:
            return 0
        # بررسی فرمت جدید
        """if data.get("status") == "1" and isinstance(data.get("result"), dict):
            return int(data["result"]["blockNumber"])

        else:
            print(f"⚠️ خطا: {data.get('message')}")
            print("پاسخ کامل:", data)
            return 0"""

    except requests.RequestException as e:
        print(f"❌ خطای ارتباطی: {e}")
        return 0


# ----- اجرای نمونه -----
YOUR_ETHERSCAN_API_KEY = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
timestamp_24h_ago = int(time.time()) - (24 * 60 * 60)
block_24h_ago = get_etherscan_block_by_timestamp(YOUR_ETHERSCAN_API_KEY,timestamp_24h_ago)
if block_24h_ago > 0:
  print(f"\n✅ شماره بلاک 24 ساعت قبل: {block_24h_ago}")



✅ شماره بلاک 24 ساعت قبل: 23885405


In [26]:
block_24h_ago

23885405

In [27]:
historical_query = f"""
    {{
      pool(id: "{pool_id}", block: {{ number: {block_24h_ago} }}) {{
        feesUSD
      }}
    }}
    """
current_query = f"""
    {{
      pool(id: "{pool_id}") {{
        feesUSD
        totalValueLockedUSD
      }}
    }}"""


In [28]:
requests.post(url, json={"query": historical_query}).json()

{'data': {'pool': {'feesUSD': '505997.769830466'}}}

In [29]:
requests.post(url, json={"query": current_query}).json()

{'data': {'pool': {'feesUSD': '507484.0935225505',
   'totalValueLockedUSD': '11763688.22625583466163655670152701'}}}

In [30]:
def calculate_public_apr(inp_pool_address: str, inp_url: str, inp_block_time_ago: int,inp_days:int):

    inp_pool_address = inp_pool_address.lower()

    # --- GraphQL query: 24 hours ago ---
    historical_query = f"""
    {{
      pool(id: "{inp_pool_address}", block: {{ number: {inp_block_time_ago} }}) {{
        feesUSD
      }}
    }}
    """

    # --- GraphQL query: current ---
    current_query = f"""
    {{
      pool(id: "{inp_pool_address}") {{
        feesUSD
        totalValueLockedUSD
      }}
    }}
    """

    # --- Request current data ---
    rjs_current = requests.post(inp_url, json={"query": current_query}).json()
    now_tvl_usd = float(find_key(rjs_current, "totalValueLockedUSD")[0])
    fee_usd_now = float(find_key(rjs_current, "feesUSD")[0])

    # --- Request historical data ---
    rjs_past = requests.post(inp_url, json={"query": historical_query}).json()
    fee_usd_time_ago = float(find_key(rjs_past, "feesUSD")[0])
    past_tvl_usd=float(find_key(rjs_current, "totalValueLockedUSD")[0])

    # --- APR Calculation ---
    daily_return = ((fee_usd_now - fee_usd_time_ago)/inp_days) / ((now_tvl_usd+past_tvl_usd)/2)
    apr = daily_return * 365

    return apr


In [31]:
calculate_public_apr(pool_address,url,block_24h_ago,1)

0.046117181718568885

In [32]:
d100_ago=time.time()-(100*24*60*60)

In [33]:
d100_ago

1755636624.2011578

In [34]:
block_100d_ago=get_etherscan_block_by_timestamp(YOUR_ETHERSCAN_API_KEY,d100_ago)
block_100d_ago

23177618

In [36]:
calculate_public_apr(pool_address,url,block_100d_ago,100)

0.11565088639991573

In [37]:
historical_query = f"""
    {{
      pool(id: "{pool_id}", block: {{ number: {block_100d_ago} }}) {{
        feesUSD
        totalValueLockedUSD
      }}
    }}
    """
current_query = f"""
    {{
      pool(id: "{pool_id}") {{
        feesUSD
        totalValueLockedUSD
      }}
    }}"""


In [38]:
requests.post(url, json={"query": historical_query}).json()

{'data': {'pool': {'feesUSD': '134749.520999973',
   'totalValueLockedUSD': '3000235.4787001707455392900337411'}}}

In [39]:
requests.post(url, json={"query": current_query}).json()

{'data': {'pool': {'feesUSD': '507484.1255713035',
   'totalValueLockedUSD': '11763691.13143561280170447050014252'}}}

In [40]:
calculate_public_apr(pool_address,url,block_100d_ago,100)

0.11565088639991573

In [41]:

# --- تنظیمات ---
pool_id = "0x5ae13baaef0620fdae1d355495dc51a17adb4082"  # استخر مورد نظر
#url = "https://gateway.thegraph.com/api/YOUR_API_KEY/subgraphs/id/YOUR_SUBGRAPH_ID"

# زمان 24 ساعت قبل به ثانیه (Unix timestamp)
time_24h_ago = int(time.time()) - 24*60*60

all_swaps = []
skip = 0
batch_size = 1000

while True:
    query = f"""
    {{
      swaps(
        where: {{
          pool: "{pool_id}",
          timestamp_gte: {time_24h_ago}
        }},
        orderBy: timestamp,
        orderDirection: asc,
        first: {batch_size},
        skip: {skip}
      ) {{
        timestamp
        amountUSD
        amount0
        amount1
      }}
    }}
    """
    r = requests.post(url, json={"query": query})
    #print(r.json())
    data = r.json().get('data', {}).get('swaps', [])

    if not data:  # اگر داده‌ای برنگشت، تمام swaps گرفته شده
        break

    all_swaps.extend(data)
    skip += batch_size

print(f"Total swaps in last 24h: {len(all_swaps)}")

# نمونه: نمایش ۵ swap اول
for swap in all_swaps[:5]:
    print(swap)


Total swaps in last 24h: 625
{'amount0': '1.57493329536439239', 'amount1': '-6554.862877', 'amountUSD': '6554.862877', 'timestamp': '1764190319'}
{'amount0': '0.022050783029663775', 'amount1': '-91.771691', 'amountUSD': '91.771691', 'timestamp': '1764190463'}
{'amount0': '-0.016599315992118884', 'amount1': '69.15272', 'amountUSD': '69.15272', 'timestamp': '1764190619'}
{'amount0': '0.29660296882374043', 'amount1': '-1234.403728', 'amountUSD': '1234.403728', 'timestamp': '1764191219'}
{'amount0': '0.286657', 'amount1': '-1192.993765', 'amountUSD': '1192.993765', 'timestamp': '1764191315'}


In [ ]:
qc=f"""

In [51]:
tick_id = f"{pool_address.lower()}#{int(current_tick)}"
print(tick_id)
q1= f"""
        {{
          tick(id: "{tick_id}") {{
            liquidityGross
            liquidityNet
          }}
        }}
        """
requests.post(url, json={"query": q1}).json()

0x5ae13baaef0620fdae1d355495dc51a17adb4082#-192998


{'data': {'tick': None}}

In [47]:
def get_tick_liquidity(pool_address: str, tick_index: int, block_number=None):
    tick_id = f"{pool_address.lower()}#{tick_index}"

    if block_number is None:
        query = f"""
        {{
          tick(id: "{tick_id}") {{
            liquidityGross
            liquidityNet
          }}
        }}
        """
    else:
        query = f"""
        {{
          tick(id: "{tick_id}", block: {{ number: {block_number} }}) {{
            liquidityGross
            liquidityNet
          }}
        }}
        """

    r = requests.post(url, json={"query": query})
    print(r.json())
    data = r.json().get("data", {}).get("tick", None)
    return data


In [48]:
get_tick_liquidity(pool_address,current_tick,block_24h_ago)

{'data': {'tick': None}}


In [61]:
# --- INPUT CONFIGURATION ---
# Replace with your actual Subgraph endpoint URL
GRAPHQL_URL = url

# Replace with your actual Pool Contract Address (e.g., PAXG/USDC)
POOL_ID = "0x5ae13baaef0620fdae1d355495dc51a17adb4082"

# Replace with the block number retrieved for 24 hours ago
BLOCK_24H_AGO = block_24h_ago

# -----------------------------------------------------------------

def execute_graphql_query(query: str, url: str) -> dict:
    """Executes a GraphQL query and returns the response dictionary."""
    try:
        response = requests.post(url, json={'query': query})
        response.raise_for_status()
        return response.json()
    except requests.RequestException:
        return {}

def get_active_liquidity_comparison(
    pool_id: str,
    block_24h_ago: int,
    graphql_url: str
) -> dict:
    """
    Fetches the active liquidity (pool.liquidity) at current and historical block.

    Returns:
        dict: {'current_liquidity': float, 'historical_liquidity': float, 'current_tick': int}
        or an empty dict if data fetching fails.
    """

    query = f"""
    query PoolLiquidityComparison {{
      current: pool(id: "{pool_id}") {{
        tick
        liquidity
      }}

      historical: pool(id: "{pool_id}", block: {{ number: {block_24h_ago} }}) {{
        liquidity
      }}
    }}
    """

    response = execute_graphql_query(query, graphql_url)

    if response and 'data' in response:
        data = response['data']
        current = data.get('current')
        historical = data.get('historical')

        if not current:
            return {}

        current_liquidity = float(current.get('liquidity', 0))
        current_tick = int(current.get('tick', 0))

        historical_liquidity = 0.0
        if historical:
            historical_liquidity = float(historical.get('liquidity', 0))

        return {
            'current_liquidity': current_liquidity,
            'historical_liquidity': historical_liquidity,
            'current_tick': current_tick
        }

    return {}

results = get_active_liquidity_comparison(
            pool_id=POOL_ID,
            block_24h_ago=BLOCK_24H_AGO,
            graphql_url=GRAPHQL_URL
        )

In [63]:
results['current_liquidity']

2.641180442045884e+18

In [64]:
results['historical_liquidity']

2.674456890003115e+18

In [65]:
import math

def calculate_position_value_in_usd(liquidity, current_tick, tick_spacing, token_decimals):
    """
    liquidity: عدد خام نقدینگی (L)
    current_tick: تیک شروع بازه
    tick_spacing: عرض بازه (مثلاً 10)
    token_decimals: تعداد اعشار توکن کوت (USDC = 6)
    """

    # 1. محاسبه مرزهای بازه (از تیک فعلی تا ۱۰ تیک بعد)
    tick_lower = current_tick
    tick_upper = current_tick + tick_spacing

    # 2. تبدیل تیک به جذر قیمت (SqrtPrice)
    # فرمول: sqrt(1.0001 ^ tick)
    sqrt_price_lower = math.sqrt(1.0001 ** tick_lower)
    sqrt_price_upper = math.sqrt(1.0001 ** tick_upper)

    # 3. محاسبه عرض بازه بر حسب قیمت
    # این نشان می‌دهد قیمت در این ۱۰ تیک چقدر تغییر می‌کند
    delta_sqrt_price = abs(sqrt_price_upper - sqrt_price_lower)

    # 4. محاسبه مقدار خام توکن (USDC)
    # فرمول اصلی: Amount = L * deltaSqrtPrice
    amount_raw = liquidity * delta_sqrt_price

    # 5. اعمال دسیمال برای تبدیل به دلار واقعی
    amount_usd = amount_raw / (10 ** token_decimals)

    return amount_usd

In [86]:
cpvc=calculate_position_value_in_usd(results['current_liquidity'],current_tick,tick_spacing,decimal1)
cpvc

85137.8578365824

In [88]:
cpvh=calculate_position_value_in_usd(results['historical_liquidity'],current_tick,tick_spacing,decimal1)
cpvh

86210.51665624816

In [78]:
all_swaps[1]

{'amount0': '0.022050783029663775',
 'amount1': '-91.771691',
 'amountUSD': '91.771691',
 'timestamp': '1764190463'}

In [79]:
all_swap_amonut=0
for i in all_swaps:
  all_swap_amonut+=float(i['amountUSD'])

In [82]:
all_swap_amonut

2972711.481674998

In [83]:
fee_tier

500.0

In [84]:
all_fees_tick=(fee_tier/10**6)*all_swap_amonut

In [85]:
all_fees_tick

1486.355740837499

In [92]:
1+all_fees_tick/(cpvc+cpvh)

1.0086744665377592

In [91]:
(1+all_fees_tick/(cpvc+cpvh))**365

23.395102959651894